# Logger Wrapper personalization/configuration

We will see on this notebook how to personalize the logger wrapper defined by sinergym.

In [10]:
import gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import (LoggerWrapper, MultiObsWrapper,
                                     NormalizeObservation)
from sinergym.utils.constants import RANGES_5ZONE

## Step 1 Inherit and modify the CSVloger

First we need to change the CSV logger to modify the values writen into the file on the funtion create_row_contents

In [11]:

from sinergym.utils.logger import CSVLogger
from typing import Any, Dict, Optional, Sequence, Tuple, Union, List

class CustomCSVLogger(CSVLogger):

    def __init__(
            self,
            monitor_header: str,
            progress_header: str,
            log_progress_file: str,
            log_file: Optional[str] = None,
            flag: bool = True):
        super(CustomCSVLogger, self).__init__(monitor_header,progress_header,log_progress_file,log_file,flag)
        self.last_10_steps_reward = [0]*10

    def create_row_contents(
            self,
            timestep: int,
            observation: List[Any],
            action: Union[List[Union[int, float]], List[None]],
            simulation_time: float,
            reward: Optional[float],
            total_power_no_units: Optional[float],
            comfort_penalty: Optional[float],
            done: bool) -> List:
        if reward is not None:
            self.last_10_steps_reward.pop(0)
            self.last_10_steps_reward.append(reward)

        return [timestep] + list(observation) + \
                       list(action) + [simulation_time, reward,np.mean(self.last_10_steps_reward),
                                       total_power_no_units, comfort_penalty, done]

    

## Step 2 Intanciate the LoggerWrapper


now we need to instantiate the loggerwrapper and specify the new headers of our file and the csvlogger class we want to use.

In [12]:
env=gym.make('Eplus-demo-v1')
env=LoggerWrapper(env,logger_class=CustomCSVLogger,monitor_header = ['timestep'] + env.variables['observation'] +
                env.variables['action'] + ['time (seconds)', 'reward', '10-mean-reward',
                'power_penalty', 'comfort_penalty', 'done'])

[2022-06-15 10:17:21,564] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-06-15 10:17:21,564] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-06-15 10:17:21,564] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf ExternalInterface object if it is not present...
[2022-06-15 10:17:21,566] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-06-15 10:17:21,566] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-06-15 10:17:21,566] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2022-06-15 10:17:21,569] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Updating idf OutPut:Variable and variables XML tree model for BVCTB connection.
[2022-06-15 10:17:21,569] E

Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

In [13]:
for i in range(1):
    obs = env.reset()
    rewards = []
    done = False
    current_month = 0
    while not done:
        a = env.action_space.sample()
        obs, reward, done, info = env.step(a)
        rewards.append(reward)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', sum(rewards), info)
    print('Episode ', i, 'Mean reward: ', np.mean(
        rewards), 'Cumulative reward: ', sum(rewards))
env.close()

[2022-06-15 10:17:23,444] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-06-15 10:17:23,444] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-06-15 10:17:23,444] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2022-06-15 10:17:23,458] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-demo-v1-res9/Eplus-env-sub_run1
[2022-06-15 10:17:23,458] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-demo-v1-res9/Eplus-env-sub_run1
[2022-06-15 10:17:23,458] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/Eplus-env-demo-v1-res9/Eplus-env-sub_run1


Reward:  -0.6725497890372522 {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'total_power': 3133.716390651042, 'total_power_no_units': -0.3133716390651042, 'comfort_penalty': -1.0317279390094, 'temperatures': [18.9682720609906], 'out_temperature': 1.8, 'action_': [17, 28]}
Reward:  -1890.290297390496 {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'total_power': 10541.15240360948, 'total_power_no_units': -1.054115240360948, 'comfort_penalty': -0.20180977695655855, 'temperatures': [19.79819022304344], 'out_temperature': -7.0, 'action_': [21, 21]}
Reward:  -3632.298944054189 {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'total_power': 984.986838734656, 'total_power_no_units': -0.09849868387346561, 'comfort_penalty': -1.7181247337670094, 'temperatures': [18.28187526623299], 'out_temperature': 8.1, 'action_': [15, 30]}
Reward:  -4781.993675772888 {'timestep': 8640, 't

[2022-06-15 10:17:38,745] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-06-15 10:17:38,745] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2022-06-15 10:17:38,745] EPLUS_ENV_demo-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
